In [1]:
import numpy as np
np.random.seed(0)

import sys
sys.path.append('../../')
import leximin_assignment

from tqdm import tqdm

import warnings

# Scratch code

In [5]:
cost_matrix = np.random.rand(5, 3)
cost_matrix

array([[0.63992102, 0.14335329, 0.94466892],
       [0.52184832, 0.41466194, 0.26455561],
       [0.77423369, 0.45615033, 0.56843395],
       [0.0187898 , 0.6176355 , 0.61209572],
       [0.616934  , 0.94374808, 0.6818203 ]])

In [6]:
capacities = np.array([1, 2, 2])

In [12]:
j = 2

smallest_agents = np.argpartition(
    cost_matrix[:, j], capacities[j] - 1
)[: capacities[j]]

smallest_agents

array([1, 2])

In [13]:
assigned_agents = []
assigned_agents += smallest_agents.tolist()
assigned_agents

[1, 2]

In [16]:
mask = [agent_id not in assigned_agents for agent_id in np.arange(0, cost_matrix.shape[0])]
mask

[True, False, False, True, True]

In [19]:
j = 1

smallest_agents = np.argpartition(
    cost_matrix[mask, j], capacities[j] - 1
)[: capacities[j]]

np.arange(0, cost_matrix.shape[0])[mask][smallest_agents]

array([0, 3])

In [23]:
assigned_agents = []
assignments = np.ones(cost_matrix.shape[0]) * 10
agent_array = np.arange(0, cost_matrix.shape[0])

for j in range(cost_matrix.shape[1] - 1, -1, -1):
    mask = [
        agent_id not in assigned_agents 
        for agent_id in agent_array
    ]
    
    filtered_smallest_agents = np.argpartition(
        cost_matrix[mask, j], capacities[j] - 1
    )[: capacities[j]]
    
    smallest_agents = agent_array[mask][filtered_smallest_agents]
    assignments[smallest_agents] = j
    assigned_agents += smallest_agents.tolist()
    
    print(j, assignments)

2 [10.  2.  2. 10. 10.]
1 [ 1.  2.  2.  1. 10.]
0 [1. 2. 2. 1. 0.]


# Test

In [2]:
N = 30
N_INTVS = 5
N_EXPERIMENTS = 100
probs = np.ones(N_INTVS) / N_INTVS

In [3]:
for _ in tqdm(range(N_EXPERIMENTS)):
    capacities = np.random.multinomial(N, probs)
    
    # cost_matrix = np.random.rand(N, N_INTVS).round(4)
    cost_matrix = np.random.randint(0, 100000, size=(N, N_INTVS))
    cost_matrix.sort(axis=1)
    
    og_lex_assigner = leximin_assignment.LeximinAssignmentHelperV4(
        cost_matrix, capacities
    )
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        assignments1 = og_lex_assigner.solve()
    
    greedy_lex_assigner = leximin_assignment.RowSortedLeximinAssignmentHelper(
        cost_matrix, capacities
    )
    assignments2 = greedy_lex_assigner.solve()
    
    if not np.array_equal(assignments1, assignments2):
        print(cost_matrix)
        print(assignments1)
        print(assignments2)
        break

100%|██████████| 100/100 [00:56<00:00,  1.78it/s]


Conclusion: no conflict

# Debugging

In [19]:
capacities

array([ 4,  9,  5, 10,  2])

In [20]:
print(assignments1)
print(assignments2.astype(int))

[1 2 2 2 3 3 0 1 3 3 1 2 3 0 1 1 3 1 4 1 0 0 3 1 3 3 3 4 2 1]
[1 2 0 2 3 3 2 1 3 3 1 2 3 0 1 1 3 1 4 1 0 0 3 1 3 3 3 4 2 1]


In [21]:
cost_matrix[np.argwhere(assignments1 != assignments2), :]

array([[[3381, 5528, 5853, 8204, 9549]],

       [[ 246, 5616, 5853, 6410, 7555]]])

In [22]:
assignments1[np.argwhere(assignments1 != assignments2)]

array([[2],
       [0]])

In [23]:
assignments2[np.argwhere(assignments1 != assignments2)]

array([[0.],
       [2.]])